In [25]:
import pandas as pd
import plotly.express as px
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine

In [26]:
def establish_connection_to_database():
    """
    Summary:
        This function establishes a connection to the database

    Arguments:
        None

    Returns:
        engine: SQLAlchemy engine object
    """

    # Load the password for the databse connection
    load_dotenv()
    DB_PW = os.getenv('DB_PW')

    # Define the connection parameters
    username = 'root'
    password = DB_PW
    host = 'localhost'
    port = '3306'
    database = 'hospital_register'

    # Create the connection
    engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}')

    return engine

In [62]:
def create_fig_hospital_numbers(engine, chart_colors):
    """
    Summary:
        This function creates a bar chart showing the number of hospitals for each provider type
    
    Arguments:
        engine (sqlalchemy.engine.base.Engine): SQLAlchemy engine object
        chart_colors (dict): Dictionary with colors for each provider type

    Returns:
        Figure: Plotly figure with the bar chart
    """

    # Write the SQL query and read the data from the database
    query = """
    SELECT pt.provider_type_name, pt.provider_type_code, COUNT(hd.hospital_id) AS num_hospitals
    FROM hospital_details hd
    INNER JOIN provider_types_dict pt ON hd.provider_type_code = pt.provider_type_code
    WHERE pt.language_code = 'en'
    GROUP BY pt.provider_type_name, pt.provider_type_code;
    """
    df_hospitals = pd.read_sql(query, engine)
    fig = px.bar(df_hospitals, x='provider_type_name', y='num_hospitals', color='provider_type_code', 
                color_discrete_map=chart_colors, height=400)
    fig.update_traces(hovertemplate="<b>%{x}</b><br>Hospital Count: %{y}<extra></extra>")
    fig.update_layout(xaxis_title='', yaxis_title='', showlegend=False)
    
    return fig

In [65]:
def create_fig_treatment_numbers(engine, chart_colors):
    """
    Summary:
        This function creates a bar chart showing the number of treatments for each provider type

    Arguments:
        engine (sqlalchemy.engine.base.Engine): SQLAlchemy engine object
        chart_colors (dict): Dictionary with colors for each provider type

    Returns:
        Figure: Plotly figure with the bar chart
    """

    # Write the SQL query and read the data from the database
    query = """
    SELECT pt.provider_type_name, pt.provider_type_code, SUM(ht.treatment_count) AS num_treatments
    FROM hospital_treatments ht
    INNER JOIN hospital_details hd ON ht.hospital_id = hd.hospital_id
    INNER JOIN provider_types_dict pt ON hd.provider_type_code = pt.provider_type_code
    WHERE pt.language_code = 'en'
    GROUP BY pt.provider_type_name, pt.provider_type_code;
    """
    df_treatments = pd.read_sql(query, engine)
    fig = px.bar(df_treatments, x='provider_type_name', y='num_treatments', color='provider_type_code', 
                color_discrete_map=chart_colors, height=400)
    fig.update_traces(hovertemplate="<b>%{x}</b><br>Treatment Count: %{y:.2s}<extra></extra>")
    fig.update_layout(xaxis_title='', yaxis_title='', showlegend=False)
    
    return fig

In [29]:
engine = establish_connection_to_database()

In [30]:
chart_colors = {
    'O': '#636EFA', # public (blue)
    'P': '#00CC96', # private (green)
    'F': '#FFA15A', # non-profit (orange)
    'darkgray': '#7F7F7F',
    'lightgray': '#e0e0e0',
    'red': '#EF553B',
    'purple': '#AB63FA',
    'yellow': '#FECB52',
}

In [63]:
fig = create_fig_hospital_numbers(engine, chart_colors)
fig.update_layout(
    plot_bgcolor='rgba(0, 0, 0, 0)',  # Transparent plot background
    paper_bgcolor='rgba(0, 0, 0, 0)',  # Transparent outer background (frame)
    margin=dict(l=0, r=0, b=0, t=0),  # No margin
    yaxis=dict(showgrid=False, zeroline=False, showline=False),  # No gridlines
)
fig.write_html("../resources/plotly_html_charts/number_of_hospitals_per_provider_type.html", config={'displayModeBar': False})

In [66]:
fig = create_fig_treatment_numbers(engine, chart_colors)
fig.update_layout(
    plot_bgcolor='rgba(0, 0, 0, 0)',  # Transparent plot background
    paper_bgcolor='rgba(0, 0, 0, 0)',  # Transparent outer background (frame)
    margin=dict(l=0, r=0, b=0, t=0),  # No margin
    yaxis=dict(showgrid=False, zeroline=False, showline=False),  # No gridlines
)
fig.write_html("../resources/plotly_html_charts/number_of_treatments_per_provider_type.html", config={'displayModeBar': False})

In [67]:
fig.show()